### Define functions (read table, make list of GCX and protein ID)
GCX means either GCA or GCF

In [187]:
import numpy as np
import pandas as pd

def read_table(file):
    df=pd.read_csv(file, sep="\t") # Read file intp script
    df=df.dropna() # Delete all enterys with 'NaN' in it
    df=df.drop(df.columns[0:1],axis=1) #'Unnamed: 0''index'
    df=df.drop_duplicates()
    df=df.sort_values(by='Organism')
    df=df.reset_index(drop=True)
    return df

# df.drop(df.columns[i], axis=1)
def get_GCX_id(table):
    GCX_id=table['GCX_id'].values.tolist() # Extract GCF IDs
    return GCX_id

def get_protein_id(table):
    proteinID=table['proteinID'].values.tolist() #Extract protein IDs
    modified_proteinID=[]
    for i in proteinID:
        new=i.split('.')
        modified_proteinID.append(new[0])
    return modified_proteinID

In [188]:
table=read_table('blastp_500_all_data_no_duplicates_sorted.txt')
# table.to_csv('blastp_500_all_data_no_duplicates_sorted.txt', sep='\t')
print(table)
GCX_id=get_GCX_id(table)
# print(GCF_id)
proteinID=get_protein_id(table)
# print(table['proteinID'].nunique())
# print(table['proteinID'])
# print(table['GCX_id'])

                                 Organism       proteinID           GCX_id
0          Acaryochloris marina MBIC11017  WP_012166727.1  GCF_000018105.1
1                 Acidobacteria bacterium      RMH16883.1  GCA_003697015.1
2                 Acidobacteria bacterium      PYT36116.1  GCA_003223075.1
3                 Acidobacteria bacterium      PYT33732.1  GCA_003223075.1
4                 Acidobacteria bacterium      PYQ38222.1  GCA_003222295.1
..                                    ...             ...              ...
478             unclassified Streptomyces  WP_093526515.1  GCF_900090115.1
479             unclassified Streptomyces  WP_103492571.1  GCF_002910775.2
480             unclassified Streptomyces  WP_099279319.1  GCF_002705975.1
481   unclassified Streptomyces(MspMP-M5)  WP_106731933.1  GCF_000373585.1
482  unclassified Streptomyces(OspMP-M45)  WP_093675912.1  GCF_900090065.1

[483 rows x 3 columns]


### Make function that finds the right files a specific folder

In [189]:
import os
def get_GCX_files(GCX_id):
    GCX_matches=[]
    for i in GCX_id:
        filename = i # This is our known part of the file name = search query
        direc = r"C:\Users\ASUS\Desktop\extract100kb_500_hits\genomic_fna" # Look in this folder
        GCX_match = [ fname for fname in os.listdir(direc) if fname.startswith(filename) ] # The search. found online. Called "search method" in the notes
        if len(GCX_match) < 1:
            GCX_matches.append('NaN')
        if len(GCX_match) == 1:
            GCX_match = ''.join(GCX_match)
            GCX_matches.append(GCX_match)
        elif len(GCX_match) > 1:
            GCX_match = ''.join(GCX_match)
            GCX_matches.append(GCX_match)            
    return GCX_matches

def get_gbprotein_files(proteinID):
    gbprotein_matches=[]
    for i in proteinID:
        filename = i # This is our known part of the file name = search query
        direc = r"C:\Users\ASUS\Desktop\extract100kb_500_hits\annotation-protein-gbfiles"
        gbprotein_match = [ fname for fname in os.listdir(direc) if fname.startswith(filename) ]
        if len(gbprotein_match) < 1:
            gbprotein_matches.append('NaN')
#             print(i)
        if len(gbprotein_match) == 1:
            gbprotein_match = ''.join(gbprotein_match)
            gbprotein_matches.append(gbprotein_match)
        elif len(gbprotein_match) > 1:
            gbprotein_match = ''.join(gbprotein_match)
            gbprotein_matches.append(gbprotein_match)
    return gbprotein_matches


In [174]:
# GCX_files=get_GCX_files(GCX_id)
# print(len(GCX_files))
# gbprotein_files=get_gbprotein_files(proteinID)
# print(len(gbprotein_files))
# # print(table[60:61])

### A function that returns the right assembly from a GCX file
It needs the GCF file and the right assembly nr (sequence_id) from the gbprotein file.

In [190]:
def find_sequence(GCX, sequence_id):
    GCX = r"C:\Users\ASUS\Desktop\extract100kb_500_hits\genomic_fna\{}".format(GCX) 
    for record in SeqIO.parse(GCX,"fasta"): # Find the right assembly version in the GCF file
#         print(record.id)
        if record.id.find(sequence_id) >= 0:
            sequence=record.seq
            return sequence
        
# test=find_sequence(GCF_files[230],'NZ_KI912153.1')
# print(test)
# print(GCF_files[number])
# print(gbprotein_files[179])

### Function that cuts the right sequence + 100kb on each side (or as much it allows)
List is None if no match between gbprotein file and assembly file. That means that it can only cut if there is a match

In [191]:
from Bio import SeqIO
def list_of_organism_seq_id_and_sequence(genbank_rec, protein_id, GCX):
    genbank_rec = r"C:\Users\ASUS\Desktop\extract100kb_500_hits\annotation-protein-gbfiles\{}".format(genbank_rec)
#     GCX = r"C:\Users\ASUS\Desktop\extract100kb_500_hits\genomic_fna\{}".format(GCF) 
    if genbank_rec == 'NaN':
        return 'gbprotein file not found'
    for rec in SeqIO.parse(genbank_rec,"gb"): # Find the locus of the protein of interest
        organism=rec.annotations['organism']
        sequence_id=rec.id
#         print(sequence_id)
        sequence=find_sequence(GCX,sequence_id)
#         print(sequence)
        if sequence != None:
            if rec.features:
#                 print(rec.features)
                for feature in rec.features:
                    if feature.type == "CDS": # Find all CDS
#                         print(feature.qualifiers['protein_id'])
                        if 'protein_id' in feature.qualifiers: # Check all fields called 'protein_id'
#                             print(feature.qualifiers['protein_id'])
                            if any(protein_id in s for s in feature.qualifiers['protein_id']): # If our protein_id is found
                                location = [int(feature.location.start),int(feature.location.end)] # Extract the location
                                strand = feature.location.strand # Find out which strand (template or compliment (+/-))
#                                 print(strand)
                                if sequence == None:
                                    return 'problem with sequence'
                                if strand == -1:
                                    if location[0]-100000 < 0 and location[1]+100000 > len(sequence):
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence.reverse_complement(),location]
                                    elif location[0]-100000 < 0:
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence[0:location[1]+100000].reverse_complement(),location] 
                                    elif location[1]+100000 > len(sequence):
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence[location[0]-100000:len(sequence)].reverse_complement(),location] 
                                    else :
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence[location[0]-100000:location[1]+100000].reverse_complement(),location] 
                                if strand == 1:
                                    if location[0]-100000 < 0 and location[1]+100000 > len(sequence):
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence.reverse_complement(),location]
                                    elif location[0]-100000 < 0:
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence[0:location[1]+100000],location] 
                                    elif location[1]+100000 > len(sequence):
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence[location[0]-100000:len(sequence)],location] 
                                    else :
                                        return [organism, sequence_id, ''.join(feature.qualifiers['protein_id']), sequence[location[0]-100000:location[1]+100000],location] 




# Main function
This ties it all together and returnes a lst of lst, which is turned into a dataframe, which holds all data (organism, assembly_nr, protein_ID,motif sequence, motif locus)

In [192]:
import itertools
import numpy as np
import pandas as pd
def all(file):
    lst_of_lst=[]
    table=read_table(file)
    table=table # Problemer med nr. 27
    GCX_id=get_GCX_id(table)
    proteinID=get_protein_id(table)
    GCX_files=get_GCX_files(GCX_id)
    gbprotein_files=get_gbprotein_files(proteinID)
    counter = 0
    for (i,j,k) in zip(gbprotein_files, proteinID, GCX_files):
        if i != 'NaN' and i != '' and k != 'NaN':
            lst=list_of_organism_seq_id_and_sequence(i, j, k)
            if lst != None:
                if len(lst)==5:
                    lst_of_lst.append(lst)
        counter += 1
    matches = len(lst_of_lst)
    return lst_of_lst

filtered_results=all('blastp_500_all_data_no_duplicates_sorted.txt')
df=pd.DataFrame.from_records(filtered_results, columns=['ORGANISM','ASSEMBLY_NR','PROTEIN_ID','M_SEQUENCE','M_LOCUS'])


In [193]:
print(df)

                           ORGANISM        ASSEMBLY_NR      PROTEIN_ID  \
0    Acaryochloris marina MBIC11017        NC_009926.1  WP_012166727.1   
1           Acidobacteria bacterium     RFGL01000382.1      RMH16883.1   
2           Acidobacteria bacterium     QHXV01000007.1      PYT36116.1   
3           Acidobacteria bacterium     QHXV01000085.1      PYT33732.1   
4           Acidobacteria bacterium     QHVP01000632.1      PYQ38222.1   
..                              ...                ...             ...   
475      Streptomyces sp. AmelKG-A3  NZ_FMYR01000011.1  WP_093526515.1   
476           Streptomyces sp. SM18      NZ_CP029342.1  WP_103492571.1   
477    Streptomyces sp. AmelKG-E11A  NZ_AQRJ01000022.1  WP_099279319.1   
478       Streptomyces sp. MspMP-M5      NZ_KB891904.1  WP_106731933.1   
479      Streptomyces sp. OspMP-M45  NZ_FLTP01000008.1  WP_093675912.1   

                                            M_SEQUENCE           M_LOCUS  
0    (C, A, A, C, A, T, T, A, T, T, 

# Write fastafile
All-in-one file + 4 files of around 100 sequences each

In [194]:
# Write file with all sequences
write_all=open(r'C:\Users\ASUS\Desktop\extract100kb_500_hits\Found_sequences\motif_plus_100kb_tails_all.fa','w')
write0_100=open(r'C:\Users\ASUS\Desktop\extract100kb_500_hits\Found_sequences\motif_plus_100kb_tails_all0_100.fa','w')
write100_200=open(r'C:\Users\ASUS\Desktop\extract100kb_500_hits\Found_sequences\motif_plus_100kb_tails_all100_200.fa','w')
write200_300=open(r'C:\Users\ASUS\Desktop\extract100kb_500_hits\Found_sequences\motif_plus_100kb_tails_all200_300.fa','w')
write300_400=open(r'C:\Users\ASUS\Desktop\extract100kb_500_hits\Found_sequences\motif_plus_100kb_tails_all300_400.fa','w')
write400_500=open(r'C:\Users\ASUS\Desktop\extract100kb_500_hits\Found_sequences\motif_plus_100kb_tails_all400_end.fa','w')

for i in df['PROTEIN_ID']: # If protein id from new result 
        write_all.write(">{}".format(i)) # Header
        write_all.write("\n")
        write_all.write(str(df.loc[df['PROTEIN_ID'] == i]['M_SEQUENCE'].values[0])) #Find row where protein ID is, and extract str(sequence).
        write_all.write("\n")
write_all.close()

for i in df['PROTEIN_ID'][0:100]: # If protein id from new result 
        write0_100.write(">{}".format(i)) # Header
        write0_100.write("\n")
        write0_100.write(str(df.loc[df['PROTEIN_ID'] == i]['M_SEQUENCE'].values[0])) #Find row where protein ID is, and extract str(sequence).
        write0_100.write("\n")
write0_100.close()

for i in df['PROTEIN_ID'][100:200]: # If protein id from new result 
        write100_200.write(">{}".format(i)) # Header
        write100_200.write("\n")
        write100_200.write(str(df.loc[df['PROTEIN_ID'] == i]['M_SEQUENCE'].values[0])) #Find row where protein ID is, and extract str(sequence).
        write100_200.write("\n")
write100_200.close()

for i in df['PROTEIN_ID'][200:300]: # If protein id from new result 
        write200_300.write(">{}".format(i)) # Header
        write200_300.write("\n")
        write200_300.write(str(df.loc[df['PROTEIN_ID'] == i]['M_SEQUENCE'].values[0])) #Find row where protein ID is, and extract str(sequence).
        write200_300.write("\n")
write200_300.close()

for i in df['PROTEIN_ID'][300:400]: # If protein id from new result 
        write300_400.write(">{}".format(i)) # Header
        write300_400.write("\n")
        write300_400.write(str(df.loc[df['PROTEIN_ID'] == i]['M_SEQUENCE'].values[0])) #Find row where protein ID is, and extract str(sequence).
        write300_400.write("\n")
write300_400.close()

for i in df['PROTEIN_ID'][400::]: # If protein id from new result 
        write400_500.write(">{}".format(i)) # Header
        write400_500.write("\n")
        write400_500.write(str(df.loc[df['PROTEIN_ID'] == i]['M_SEQUENCE'].values[0])) #Find row where protein ID is, and extract str(sequence).
        write400_500.write("\n")
write400_500.close()



# Stats
missing_data = read data from file, make a df with all unique protein ids, deleted from read_table. <br>

In [197]:
df2=pd.read_csv('blastp_500_all_data_no_duplicates_sorted.txt', sep="\t")

def missing_data(file):
    df=pd.read_csv(file, sep="\t")
    df1=df[df.isna().any(axis=1)] # Find all data with NaN
    new_df=df1.dropna(subset=['proteinID']) # Delete all entries, where NaN is present in protein ID.
    unique=new_df['proteinID'].unique()
    del new_df['Unnamed: 0']
    return new_df

missing=missing_data('blastp_500_all_data_no_duplicates_sorted.txt')

proteinID_not_found=[]
for i in table['proteinID'].values: # If protein ID is used in the script
    if i not in df['PROTEIN_ID'].values and i not in proteinID_not_found: # But not found in the results  and i not in proteinID_not_found
        proteinID_not_found.append(i)

In [198]:
print("ProteinIDs from Daniela's table:",df2['proteinID'].nunique())
print("ProteinIDs with no GCFid",'({})'.format(len(missing))) #print(missing)
print(missing)
print("ProteinIDs with problems", len(proteinID_not_found))
print(proteinID_not_found)
print("ProteinIDs found", df['PROTEIN_ID'].nunique())
# print(table.loc[table['proteinID'] =='HAI15717.1'])

ProteinIDs from Daniela's table: 493
ProteinIDs with no GCFid (10)
                                           Organism       proteinID GCX_id
46                            Bacillus sp. NSP2.1**  WP_026557213.1    NaN
47                            Bacillus sp. NSP2.1**  WP_026557206.1    NaN
82                       Candidatus Kentron sp. FW*      VFJ75372.1    NaN
112                  Deltaproteobacteria bacterium*      HEB89883.1    NaN
157                  Gammaproteobacteria bacterium*      HEB56032.1    NaN
225            Micromonospora purpureochromogenes**  WP_030498975.1    NaN
226            Micromonospora purpureochromogenes**  WP_030502634.1    NaN
277                             Myxococcus fulvus**  WP_046711868.1    NaN
302  Nostoc sp. 'Peltigera membranacea cyanobiont'*      ADA69241.1    NaN
337             Planktothrix prolifica NIVA-CYA 98*      CAQ48282.1    NaN
ProteinIDs with problems 3
['MAG31691.1', 'CUM62319.1', 'ACG63859.1']
ProteinIDs found 480
